In [6]:
# Imports
import numpy as np
import itertools

Declacaration of the Euclidean Space $\mathbb{R}^3$

In [7]:
M = EuclideanSpace(3, 'M', start_index=1)
X.<x,y,z> = M.chart()

eX = X.frame()
dX = X.coframe()



Declaration of a 0-form in $\mathbb{R}^3$ (scalar field)

In [ ]:
f = M.scalar_field(x^2+y^2+z^2)
f.display()

M → ℝ
(x, y, z) ↦ x^2 + y^2 + z^2

Declaration of $\omega \in \Omega^1(\mathbb{R}^3)$

See Example 3 in the paper

In [9]:
omega = M.diff_form(1, {eX:[x,y,z]}, name='a')
omega.display(X)

a = x dx + y dy + z dz

Declaration of $\omega' \in \Omega^2(\mathbb{R}^3)$

See Example 4 in the paper

In [11]:
omegap = M.diff_form(2, name='b')
omegap[eX, 1, 2] = 0 #dx^dy
omegap[eX, 1, 3] = 0 #dx^dz
omegap[eX, 2, 3] = x #dy^dz
omegap.display(X)

b = x dy∧dz

# Black box sampler used to obtain numerical values of a $k$-form in $\mathbb{R}^3$ at a given point
See Algorithm 1 in the paper

In [12]:
def numerical_sampler_kform(M, k_form, point):
    """
    Sample numerical values of a k-form at a point
    :param M: manifold
    :param k_form: k-form to sample
    :param point: point at which to sample
    :return: numerical values of the k-form at the point as a tensor
    """
    
    # Check if the k-form is a 0-form (scalar field)
    if k_form.degree() == 0: 
        p = M.point(point)
        return np.array(float(k_form(p)))
    
    if k_form.degree() == 1:
        p = M.point(point)
        X.<x,y,z> = M.chart()
        eX = X.frame()
        return np.array([float(k_form(eX[i])(p)) for i in range(1,M.dimension()+1)])
    
    if k_form.degree() == 2:
        p = M.point(point)
        X.<x,y,z> = M.chart()
        eX = X.frame()
        return np.array([[float(k_form(eX[i], eX[j])(p)) for j in range(1,M.dimension()+1)] for i in range(1,M.dimension()+1)])
    
    if k_form.degree() == 3:
        p = M.point(point)
        X.<x,y,z> = M.chart()
        eX = X.frame()
        return np.array([[[float(k_form(eX[i], eX[j], eX[k])(p)) for k in range(1,M.dimension()+1)] for j in range(1,M.dimension()+1)] for i in range(1,M.dimension()+1)])

# Numerical exterior derivative in $\mathbb{R}^3$
See Algorithm 2 in the paper

In [14]:
def numerical_exterior_derivative(M, k_form, point, epsilon=1e-5):
    """
    Sample numerical values of the exterior derivative of a k-form at a point
    :param M: manifold
    :param k_form: k-form to sample
    :param point: point at which to sample
    :return: numerical values of the exterior derivative of the k-form at the point as a tensor
    """
    
    # If the k-form is a 0-form (scalar field)
    if k_form.degree() == 0: 
        dw = np.zeros(3)
        directions = [np.array(np.eye(3, dtype=int)[i]) for i in range(3)]
        point = np.array(point)
        for i in range(3):
            dw[i] = (1/(2*epsilon))*(numerical_sampler_kform(M, k_form, point + epsilon*directions[i]) - numerical_sampler_kform(M, k_form, point - epsilon*directions[i]))
        
        return dw
    
    # If the k-form is a 1-form
    elif k_form.degree() == 1:
        dw = np.zeros((3,3))
        directions = [np.array(np.eye(3, dtype=int)[i]) for i in range(3)]
        point = np.array(point)
        for i in range(3):
            for j in range(3):
                if i!=j:
                    dw[i,j] = (1/(2*epsilon))*(numerical_sampler_kform(M, k_form, point -  epsilon*directions[j])[i] +\
                                            numerical_sampler_kform(M, k_form, point +  epsilon*directions[i])[j] -\
                                            numerical_sampler_kform(M, k_form, point +  epsilon*directions[j])[i] -\
                                            numerical_sampler_kform(M, k_form, point -  epsilon*directions[i])[j])
        
        return dw
    
    # If the k-form is a 2-form
    elif k_form.degree() == 2:
        dw = np.zeros((3,3,3))
        directions = [np.array(np.eye(3, dtype=int)[i]) for i in range(3)]
        point = np.array(point)
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    if i!=j and j!=k and i!=k:
                        dw[i,j,k] = (1/(2*epsilon))*(+numerical_sampler_kform(M, k_form, point +  epsilon*directions[i])[j,k] \
                                            -numerical_sampler_kform(M, k_form, point -  epsilon*directions[i])[j,k] \
                                            -numerical_sampler_kform(M, k_form, point +  epsilon*directions[j])[i,k] \
                                            +numerical_sampler_kform(M, k_form, point -  epsilon*directions[j])[i,k] \
                                            +numerical_sampler_kform(M, k_form, point +  epsilon*directions[k])[i,j] \
                                            -numerical_sampler_kform(M, k_form, point -  epsilon*directions[k])[i,j])
                                            
        return dw
    
    elif k_form.degree() >= 3:
        assert False, "Not defined in 3D space."
        
        
        

# Usage examples

## Example 3 in the paper:

In [15]:
numerical_sampler_kform(M, omega, (1,1,1))

array([1., 1., 1.])

In [16]:
numerical_exterior_derivative(M, omega, (1,1,1))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

## Example 4 in the paper: 

In [17]:
numerical_sampler_kform(M, omegap, (1,2,3))

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0., -1.,  0.]])

In [18]:
numerical_exterior_derivative(M, omegap, (1,2,3)) 

array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  1.],
        [ 0., -1.,  0.]],

       [[ 0.,  0., -1.],
        [ 0.,  0.,  0.],
        [ 1.,  0.,  0.]],

       [[ 0.,  1.,  0.],
        [-1.,  0.,  0.],
        [ 0.,  0.,  0.]]])